Write a Single Python script that integrates OpenAI GPT-4, Anthropic Claude, and Google Gemini into one app. It uses a router function to choose the model dynamically.
# multi_llm_router.py

import os
import openai
import anthropic
import google.generativeai as genai

# --- Set up API keys ---
openai.api_key = os.getenv("OPENAI_API_KEY")
anthropic_key = os.getenv("ANTHROPIC_API_KEY")
google_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=google_key)

# --- GPT-4 ---
def call_gpt4(prompt: str) -> str:
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

# --- Claude (Sonnet or Opus) ---
def call_claude(prompt: str) -> str:
    client = anthropic.Anthropic(api_key=anthropic_key)
    response = client.messages.create(
        model="claude-3-sonnet-20240229",  # or use "claude-3-opus-20240229"
        max_tokens=300,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.content[0].text

# --- Gemini ---
def call_gemini(prompt: str) -> str:
    model = genai.GenerativeModel("gemini-pro")
    response = model.generate_content(prompt)
    return response.text

# --- Unified Router ---
def call_llm(model_name: str, prompt: str) -> str:
    if model_name == "gpt4":
        return call_gpt4(prompt)
    elif model_name == "claude":
        return call_claude(prompt)
    elif model_name == "gemini":
        return call_gemini(prompt)
    else:
        return "Invalid model name. Choose from: gpt4, claude, gemini."

# --- Example usage ---
if __name__ == "__main__":
    model = input("Enter model (gpt4 / claude / gemini): ").strip().lower()
    prompt = input("Enter your prompt: ")

    result = call_llm(model, prompt)
    print("\n🧠 LLM Response:\n", result)


# 🤖 LLM Response Comparison and Scoring Script

```python
import os
import openai
import anthropic
import google.generativeai as genai

# --- Configure API keys ---
openai.api_key = os.getenv("OPENAI_API_KEY")
anthropic_key = os.getenv("ANTHROPIC_API_KEY")
google_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=google_key)

# --- GPT-4 ---
def call_gpt4(prompt: str) -> str:
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

# --- Claude ---
def call_claude(prompt: str) -> str:
    client = anthropic.Anthropic(api_key=anthropic_key)
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=300
    )
    return response.content[0].text

# --- Gemini ---
def call_gemini(prompt: str) -> str:
    model = genai.GenerativeModel("gemini-pro")
    response = model.generate_content(prompt)
    return response.text

# --- Scoring function ---
def score_response(response: str) -> int:
    score = 0
    if len(response) > 100:
        score += 1
    if any(word in response.lower() for word in ["however", "therefore", "in conclusion"]):
        score += 1
    if "?" in response:
        score += 1
    return score

# --- Compare and score all models ---
def compare_models(prompt: str):
    print(f"Prompt: {prompt}\n{'='*50}")

    models = {
        "GPT-4": call_gpt4(prompt),
        "Claude": call_claude(prompt),
        "Gemini": call_gemini(prompt)
    }

    scores = {}
    for name, response in models.items():
        score = score_response(response)
        scores[name] = {"score": score, "response": response}

    # Sort and display results
    sorted_scores = sorted(scores.items(), key=lambda x: x[1]['score'], reverse=True)

    for name, result in sorted_scores:
        print(f"\n### 🧠 {name} — Score: {result['score']}/3")
        print(result["response"])
        print("\n" + "-"*80)

# --- Run comparison ---
if __name__ == "__main__":
    prompt = "Explain black holes to a 10-year-old."
    compare_models(prompt)


# 🤖 Techniques to Orchestrate Between Different AI Models

Orchestration means coordinating multiple AI models to work together efficiently, based on their strengths.

---

## ✅ 1. Routing Based on Task Type

Use different models for different types of tasks.

```python
def route_by_task(task_type, prompt):
    if task_type == "code":
        return call_gpt4(prompt)
    elif task_type == "summary":
        return call_claude(prompt)
    elif task_type == "vision":
        return call_gemini(prompt)


## ✅ 2. Fallback or Redundancy Logic

Use a backup model if the primary one fails or provides a low-quality response.

```python
def call_with_fallback(prompt):
    try:
        return call_claude(prompt)
    except:
        return call_gpt4(prompt)


## ✅ 3. Model Voting or Evaluation

Send the same prompt to multiple models and compare their responses using a scoring function or evaluation logic.

```python
responses = {
    "gpt4": call_gpt4(prompt),
    "claude": call_claude(prompt),
    "gemini": call_gemini(prompt)
}

def score_response(resp):
    score = 0
    if "because" in resp.lower():
        score += 1
    if len(resp) > 100:
        score += 1
    if "however" in resp.lower():
        score += 1
    return score

best_model = max(responses, key=lambda x: score_response(responses[x]))
print(f"✅ Best model: {best_model}")
print(responses[best_model])


## ✅ 4. Chain of Models (Sequential Pipeline)

Use the output of one model as the input to another — useful for multi-step reasoning, document processing, or creativity pipelines.

### 🔗 Example:

```python
# Step 1: Gemini extracts key information from an article
facts = call_gemini("Extract 3 key facts from this article:\n" + article_text)

# Step 2: Claude rewrites the facts in simpler language
simplified = call_claude("Rewrite the following facts simply:\n" + facts)

# Step 3: GPT-4 turns the simplified version into a tweet
tweet = call_gpt4("Create a tweet based on these points:\n" + simplified)

print("✅ Final Tweet:\n", tweet)



## ✅ 5. Multi-Agent Workflow

Use multiple AI models as specialized agents that collaborate on a task. Each model plays a role such as **planner**, **executor**, or **reviewer** — similar to a team working together.

---

### 👥 Example: Blog Writing Workflow

```python
# Claude as the Planner — creates an outline
outline = call_claude("Create an outline for a blog post on 'The Future of AI in Education'.")

# GPT-4 as the Writer — expands the outline into a full article
blog_post = call_gpt4("Write a blog post based on this outline:\n" + outline)

# Gemini as the Reviewer — provides feedback and edits
review = call_gemini("Review this blog post and suggest improvements:\n" + blog_post)

# Display results
print("📝 Blog Post:\n", blog_post)
print("\n🔍 Reviewer Feedback:\n", review)


In [ ]:
# multi_model_orchestration.py

import os
import openai
import anthropic
import google.generativeai as genai

# === API Keys (Set your keys as environment variables) ===
openai.api_key = os.getenv("OPENAI_API_KEY")
anthropic_key = os.getenv("ANTHROPIC_API_KEY")
google_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=google_key)

# === Model Call Functions ===

def call_gpt4(prompt: str) -> str:
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

def call_claude(prompt: str) -> str:
    client = anthropic.Anthropic(api_key=anthropic_key)
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=300,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.content[0].text.strip()

def call_gemini(prompt: str) -> str:
    model = genai.GenerativeModel("gemini-pro")
    response = model.generate_content(prompt)
    return response.text.strip()

# === Step 1: Routing Based on Task Type ===

def route_by_task(task_type: str, prompt: str) -> str:
    if task_type == "code":
        return call_gpt4(prompt)
    elif task_type == "summary":
        return call_claude(prompt)
    elif task_type == "vision":
        return call_gemini(prompt)
    else:
        return "Invalid task type."

# === Step 2: Fallback Logic ===

def call_with_fallback(prompt: str) -> str:
    try:
        return call_claude(prompt)
    except:
        return call_gpt4(prompt)

# === Step 3: Voting / Scoring ===

def score_response(response: str) -> int:
    score = 0
    if "because" in response.lower():
        score += 1
    if len(response) > 100:
        score += 1
    if "however" in response.lower():
        score += 1
    return score

def model_voting(prompt: str) -> str:
    responses = {
        "GPT-4": call_gpt4(prompt),
        "Claude": call_claude(prompt),
        "Gemini": call_gemini(prompt)
    }

    for model, resp in responses.items():
        print(f"\n--- {model} ---\n{resp}\nScore: {score_response(resp)}")

    best_model = max(responses, key=lambda x: score_response(responses[x]))
    return f"\n✅ Best model: {best_model}\nResponse:\n{responses[best_model]}"

# === Step 4: Chain of Models ===

def model_chain(article_text: str) -> str:
    # Gemini extracts key facts
    facts = call_gemini("Extract 3 key facts from this article:\n" + article_text)
    print("\n🔹 Gemini Facts:\n", facts)

    # Claude rewrites them simply
    simplified = call_claude("Rewrite the following simply:\n" + facts)
    print("\n🔹 Claude Simplified:\n", simplified)

    # GPT-4 turns them into a tweet
    tweet = call_gpt4("Create a tweet using these facts:\n" + simplified)
    return f"\n✅ Final Tweet:\n{tweet}"

# === Step 5: Multi-Agent Workflow ===

def multi_agent_blog_workflow(topic: str) -> str:
    # Claude = planner
    outline = call_claude(f"Create a blog post outline on: {topic}")
    print("\n🧠 Claude (Planner) Outline:\n", outline)

    # GPT-4 = writer
    blog_post = call_gpt4("Write a blog post using this outline:\n" + outline)
    print("\n✍️ GPT-4 (Writer) Blog Post:\n", blog_post)

    # Gemini = reviewer
    review = call_gemini("Review this blog post and provide suggestions:\n" + blog_post)
    return f"\n✅ Gemini (Reviewer) Feedback:\n{review}"

# === MAIN ===

if __name__ == "__main__":
    prompt = "Explain quantum computing in simple terms."
    article = "Quantum computers use quantum bits or qubits which can be in superpositions, enabling them to solve certain problems exponentially faster than classical computers."

    print("\n=== STEP 1: Routing ===")
    print(route_by_task("summary", prompt))

    print("\n=== STEP 2: Fallback ===")
    print(call_with_fallback(prompt))

    print("\n=== STEP 3: Voting / Evaluation ===")
    print(model_voting(prompt))

    print("\n=== STEP 4: Chain of Models ===")
    print(model_chain(article))

    print("\n=== STEP 5: Multi-Agent Workflow ===")
    print(multi_agent_blog_workflow("The Future of AI in Education"))


# how to switch between cloud AI providers (OpenAI and Claude in this case) using a unified interface with minimal code changes.

# 🔄 Full Python Code: Switch Between AI Providers (OpenAI & Claude)

This code defines a common interface and lets you switch providers easily by changing a single line or config.

---

## 🧠 `ai_provider.py`

```python
import os
import openai
import anthropic

# === Common Interface ===
class AIProvider:
    def generate(self, prompt: str) -> str:
        raise NotImplementedError("Must implement 'generate' method")

# === OpenAI GPT-4 Adapter ===
class OpenAIProvider(AIProvider):
    def generate(self, prompt: str) -> str:
        openai.api_key = os.getenv("OPENAI_API_KEY")
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content.strip()

# === Claude Adapter (Anthropic) ===
class ClaudeProvider(AIProvider):
    def __init__(self):
        self.client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

    def generate(self, prompt: str) -> str:
        response = self.client.messages.create(
            model="claude-3-sonnet-20240229",
            max_tokens=300,
            messages=[{"role": "user", "content": prompt}]
        )
        return response.content[0].text.strip()

# === Provider Selector (Factory) ===
def get_provider(name: str) -> AIProvider:
    if name == "openai":
        return OpenAIProvider()
    elif name == "claude":
        return ClaudeProvider()
    else:
        raise ValueError("Unsupported AI provider: " + name)


# main code 

from ai_provider import get_provider
import os

# Choose provider via config/env
provider_name = os.getenv("AI_PROVIDER", "openai")  # change to "claude" to switch
provider = get_provider(provider_name)

# Prompt to run
prompt = "Explain the difference between AI and Machine Learning."

# Get response
response = provider.generate(prompt)

# Output
print(f"\n🤖 Provider: {provider_name.capitalize()}")
print("📝 Response:\n", response)


✅ Benefits of This Design

Feature	Advantage
🔄 Easily Switch	Change one line or env var
🧱 Clean Code	Common interface, no duplicate logic
✨ Extensible	Add more providers (e.g., Gemini, Cohere) easily
🧪 Testable	Each provider can be mocked/tested independently

